In [ ]:
# 韵律特征
import os
import pandas as pd

path = "/home/wd/data/NCMMSC/prosody"

train_set = pd.read_csv(os.path.join(path, "prosody_train.csv"))
test_set = pd.read_csv(os.path.join(path, "prosody_test.csv"))

train_set.sort_values(by="AudioFile", inplace=True)
test_set.sort_values(by="AudioFile", inplace=True)
train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)

In [ ]:
# proprecessing 删去AD_F_040108与HC_F_122302
name = train_set["AudioFile"]
drop_idx = [i for i, n in enumerate(name) if (n.startswith("AD_F_040108") or n.startswith("HC_F_122302"))]

train_set.drop(drop_idx, inplace=True)
name = train_set["AudioFile"].tolist()
train_set.drop("AudioFile", axis=1, inplace=True)

train_set

train_set.reset_index(inplace=True, drop=True)

In [ ]:
train_label = [i.split("_")[0] for i in name]
test_label = [i.split("_")[1][:-4] for i in test_set["AudioFile"].values]
test_set.drop("AudioFile", axis=1, inplace=True)

In [ ]:
train_prosody, test_prosody = train_set.copy(), test_set.copy()

In [ ]:
train_set = pd.read_csv("/home/wd/data/SystemDemo/trainset_is.csv")
train_set.sort_values(by="name", inplace=True)
train_set.reset_index(inplace=True, drop=True)

In [ ]:
# proprecessing 删去AD_F_040108与HC_F_122302
drop_idx = [i for i, n in enumerate(train_set["name"]) if (n.startswith("AD_F_040108") or n.startswith("HC_F_122302"))]

train_set.drop(drop_idx, inplace=True)
train_set.reset_index(inplace=True, drop=True)

train_set

In [ ]:
test_set = pd.read_csv("/home/wd/data/SystemDemo/testset_is.csv")
test_set.sort_values(by="name", inplace=True)
test_set.reset_index(inplace=True, drop=True)
test_set

In [ ]:
train_ege, test_ege = train_set.copy(), test_set.copy()

In [ ]:
# 拼接两组特征
train_combine = pd.concat([train_ege, train_prosody], axis=1)
test_combine = pd.concat([test_ege, test_prosody], axis=1)

## PCA + AVONA + SVM 调参

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
clf = make_pipeline(
    StandardScaler(),
    PCA(n_components=0.8),
    SelectKBest(f_classif, k=4),
    SVC(kernel="rbf", class_weight={"HC":1, "MCI":1, "AD":2}, C=5)
)
clf.fit(train_combine.iloc[:,2:], train_label)
pred = clf.predict(test_combine.iloc[:, 2:])

In [ ]:
print(accuracy_score(test_label, pred))
print(precision_score(test_label, pred, average="weighted"))
# print(roc_auc_score(test_label, pred))
print(recall_score(test_label, pred, average="weighted"))
print(f1_score(test_label, pred, average="weighted"))

In [ ]:
import joblib

joblib.dump(clf, 'svm.model')